**NOTE:**  This notebook was run on a kaggle kernel for the use of the free gpu and openCV library which I didn't care to install locally.

This notebook was run as a data pipeline with outputs from here used as inputs for the speed processing notebook

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# video to flow video

In [ ]:
def video_to_flow_video(vid_path, flow_path):
    reader = cv2.VideoCapture(vid_path)
    writer = cv2.VideoWriter(flow_path, cv2.VideoWriter_fourcc(*'mp4v'), 20.0, (640, 250))
    i=0
    prv = None

    while(True):
        ret, frame = reader.read()
        if not ret:
            print(f"Can't receive frame: {i} (stream end?).")
            break  # end of video

        frame = frame[50:300]  # crop dash and roof
        nxt = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

        if prv is not None:
            flow = cv2.calcOpticalFlowFarneback(prv,nxt, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])

            hsv = np.zeros_like(frame)
            hsv[...,0] = ang*180/np.pi/2  # set hue
            hsv[...,1] = 255  # max saturation
            hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)  # norm values

            bgr = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
            writer.write(bgr)
            #plt.imshow(bgr)

            if i%100==0:print('.', end='')

        prv = nxt
        i+=1

    reader.release()
    writer.release()

In [ ]:
# video_to_flow_video("/kaggle/input/train.mp4", "/kaggle/working/train_flow.mp4")
# video_to_flow_video("/kaggle/input/test.mp4", "/kaggle/working/test_flow.mp4")

# video to flow frames

In [ ]:
def video_to_flow_images(vid_path, img_path):
    if not os.path.exists(img_path): os.makedirs(img_path)

    reader = cv2.VideoCapture(vid_path)
    i=0
    prv = None

    while(True):
        ret, frame = reader.read()
        if not ret:
            print(f"Can't receive frame: {i} (stream end?).")
            break  # end of video

        frame = frame[50:300]  # crop dash and roof
        nxt = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

        if prv is not None:
            flow = cv2.calcOpticalFlowFarneback(prv,nxt, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])

            hsv = np.zeros_like(frame)
            hsv[...,0] = ang*180/np.pi/2  # set hue
            hsv[...,1] = 255  # max saturation
            hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)  # norm values

            rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2RGB)
            cv2.imwrite(img_path+"{:05n}".format(i)+'.png',rgb)

            if i%100==0:print('.', end='')

        prv = nxt
        i+=1
        
    reader.release()

In [ ]:
video_to_flow_images("/kaggle/input/train.mp4", "/kaggle/working/train/")
video_to_flow_images("/kaggle/input/test.mp4", "/kaggle/working/test/")